# Run Qwen PPO with [verl](https://github.com/volcengine/verl)

This tutorial provides a step-by-step guide to using verl for executing your RLHF pipeline. You can find our [github repo](https://github.com/volcengine/verl/) and [documentation](https://verl.readthedocs.io/en/latest/index.html) for mode details.

This notebook is also published on the [Lightning Studio](https://lightning.ai/hlin-verl/studios/verl-getting-started) platform, which provides free GPU quota every month. Checkout the published notebook with pre-installed dependencies using a free L4 GPU [here](https://lightning.ai/hlin-verl/studios/verl-getting-started) (no credit card required).

### You will learn:

- How to install verl from scratch.
- How to use existing scripts to run an RLHF pipeline with your own models and data.

# Dependency Installation

If you are running on Lightning Studio using the published notebook, the dependencies are **already installed** and you can proceed to step "**Load Pretrained Language Model**"

In [3]:
!pip install --upgrade pip setuptools wheel
!pip install torch==2.4.0 torchvision==0.19.0
!pip list | grep torch
# # uv pip install flash-attn --no-build-isolationwarp_2020

torch                             2.4.0
torchaudio                        2.6.0
torchdata                         0.11.0
torchvision                       0.19.0


In [4]:
!export RAY_TMPDIR=/data/ray
!ray start --head  --include-dashboard=False --port=6380

# !ray start --help


Usage stats collection is disabled.

Local node IP: 10.36.16.97

--------------------
Ray runtime started.
--------------------

Next steps
  To add another node to this Ray cluster, run
    ray start --address='10.36.16.97:6380'
  
  To connect to this Ray cluster:
    import ray
    ray.init()
  
  To terminate the Ray runtime, run
    ray stop
  
  To view the status of the cluster, use
    ray status


In [10]:
!ray start 

`--address` is a required flag unless starting a head node with `--head`.


## Install and verify verl
Now we're ready to install verl!

In [5]:
# # In case you run this notebook and have not cloned verl yet:
!git clone https://github.com/volcengine/verl $HOME/verl_repo

!cd $HOME/verl_repo && pip3 install -e . -U

fatal: destination path '/home/warp/verl_repo' already exists and is not an empty directory.
Obtaining file:///home/warp/verl_repo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for verl (pyproject.toml) ... done
  Created wheel for verl: filename=verl-0.2.0.dev0-0.editable-py3-none-any.whl size=15073 sha256=5cb9d83df3b450bbc5ea2b17139cb9ac64263a992aa1db1a32151272a4d542a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-dxo3ngpq/wheels/74/95/39/c6a2bfeb9bb59ae258557c3e8f5dd6e1479e797790335e5601
Successfully built verl
  Attempting uninstall: verl
    Found existing installation: verl 0.2.0.dev0
    Uninstalling verl-0.2.0.dev0:
      Successfully uninstalled verl-0.2.0.dev0


## Restart the python kernel

In [ ]:
import IPython

# # Restart the kernel to pickup the latest python packages
IPython.get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

: 

In [6]:
import ray
ray.init(_temp_dir="/data/ray", )

2025-05-14 12:59:26,234	INFO worker.py:1879 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.9
Ray version:,2.45.0
Dashboard:,http://127.0.0.1:8265


In [8]:
ray.shutdown()
ray.init(_plasma_directory="/data/ray", include_dashboard=False)


2025-05-14 13:00:01,786	INFO worker.py:1694 -- Connecting to existing Ray cluster at address: 10.36.16.97:6380...


KeyboardInterrupt: 

In [9]:
rm -rf /tmp/ray/*

In [10]:
import torch
try:
    assert torch.cuda.is_available() is True
    torch.ones(1, dtype=torch.bfloat16).cuda()
except AssertionError:
    print("Please switch to an env with GPUs supporting bfloat16 (L4 RTX 5000, A5000, A100, H100, A10, etc)")

try:
    import verl
except Exception as e:
    print("Please install verl via pip and restart the kernel")
    raise e

import flash_attn


# Load Pretrained Language Model

verl supports models available in Huggingface transformers (as well as custom Megatron models).

Let's download the model first.

In [11]:
!huggingface-cli download vichitrarora/qwen_finetuned_sft --local-dir $HOME/models/qwen_finetuned_sft

# If huggingface-cli is not stable, use the method below
# import transformers
# transformers.pipeline('text-generation', model='Qwen/Qwen2.5-0.5B-Instruct')

Fetching 11 files: 100%|███████████████████████| 11/11 [00:00<00:00, 911.93it/s]
/home/warp/models/qwen_finetuned_sft


# Dataset preparation

We train with the Grade School Math 8K (GSM8k) task in this demo. The dataset is downloaded from huggingface [gsm8k](https://huggingface.co/datasets/openai/gsm8k) and below are some samples:


**Prompt**

Katy makes coffee using teaspoons of sugar and cups of water in the ratio of 7:13. If she used a total of 120 teaspoons of sugar and cups of water, calculate the number of teaspoonfuls of sugar she used.

**Solution**

The total ratio representing the ingredients she used to make the coffee is 7+13 = <<7+13=20>>20 Since the fraction representing the number of teaspoons she used is 7/20, she used 7/20120 = <<7/20120=42>>42 #### 42

# the reward

We use a rule-based reward model. We force the model to produce a final answer following 4 `#` as shown in the solution. We extract the final answer from both the solution and model's output using regular expression matching. We compare them and assign a reward of 1 to correct answer, 0.1 to incorrect answer and 0 to no answer.

In [12]:
import inspect
from reward import your_reward_function as gsm8k_reward

print(inspect.getsource(gsm8k_reward))

def your_reward_function(data_source, solution_str, ground_truth, extra_info=None):
    """
    Integrated reward function which first extracts the MongoDB query from the model's output,
    then computes various scores (syntax, schema mapping, keyword similarity, and query similarity).
    
    Args:
        solution_str (str): The raw model output containing the MongoDB query.
        ground_truth (str): The ground_truth MongoDB query.
        method (str): Evaluation method (not directly used here).
        format_score (float): Score to return if not perfect.
        score (float): Score to return if perfect.
    
    Returns:
        float: The final reward score.
    """
    # Extract the actual MongoDB query from the solution_str.
    extracted_solution_str = extract_mongodb_query(solution_str)
    if extracted_solution_str is None:
        return 0

    syntax_score = check_mongodb_syntax(extracted_solution_str, is_predicted=True)
    schema_mapping_score = compare_queries(extr

# Run the RL Pipeline
Let's start with the Proximal Policy Optimization (PPO) algorithm,  one of the most widely used methods for post-training large language models.

The main entry point of the PPO algorithm example is: `main_ppo.py`. A detailed guide to understanding the code architecture of `main_ppo.py` is available [here](https://verl.readthedocs.io/en/latest/examples/ppo_code_architecture.html).

In this tutorial, we will demonstrate how to run the PPO algorithm with **Qwen 2.5-0.5B** by setting:
- `trainer.n_gpus_per_node`: Number of GPUs per node.

- `actor_rollout_ref.rollout.tensor_model_parallel_size`: TP size for rollout. Only effective for vllm.

- `actor_rollout_ref/critic.model.path`: Huggingface model path. This can be either local path or HDFS path. For HDFS path, we provide utils to download it to DRAM and convert the HDFS path to local path.

- `data.train_batch_size`: Batch size sampled for one training iteration of different RL algorithms.

- `data.max_prompt_length`: Maximum prompt length. All prompts will be left-padded to this length. An error will be reported if the length is too long.

- `data.max_response_length`: Maximum response length. Rollout in RL algorithms (e.g. PPO) generates up to this length.

- `actor_rollout_ref.actor.ppo_mini_batch_size`: One sample is split into multiple sub-batches with batch_size=ppo_mini_batch_size for PPO updates.

- `actor_rollout_ref/critic.actor.ppo_micro_batch_size`: Similar to gradient accumulation, the micro_batch_size for one forward pass, trading speed for GPU memory.

The full configuration explanation is available [here](https://verl.readthedocs.io/en/latest/examples/config.html).

The training may take a few hours to finish but you can observe how the model performance increases. It will progressively output:

- generated sentences.

- step information with RL metrics, such as entropy loss, kl, and ``val/test_score/openai/gsm8k`` (validated every ``trainer.test_freq`` steps)

If you come across GPU out of memory issues, set smaller values for the micro batch size used for gradient accumulation:

- actor_rollout_ref.actor.ppo_micro_batch_size=1
- critic.ppo_micro_batch_size=1

In [ ]:


!PYTHONUNBUFFERED=1 python3 /home/warp/metafusion/main_ppo.py\
 data.train_files=/home/warp/metafusion/train.parquet \
 data.val_files=/home/warp/metafusion/eval.parquet \
 data.train_batch_size=64 \
 data.max_prompt_length=1024 \
 data.max_response_length=512 \
 actor_rollout_ref.model.path=vichitrarora/qwensftfinetuned \
 actor_rollout_ref.actor.optim.lr=1e-6 \
 actor_rollout_ref.actor.ppo_mini_batch_size=16 \
 actor_rollout_ref.rollout.log_prob_micro_batch_size_per_gpu=1 \
 actor_rollout_ref.rollout.tensor_model_parallel_size=1 \
 actor_rollout_ref.rollout.gpu_memory_utilization=0.4 \
 actor_rollout_ref.ref.log_prob_micro_batch_size_per_gpu=4 \
 critic.optim.lr=1e-5 \
 critic.model.path=vichitrarora/qwensftfinetuned \
 critic.ppo_micro_batch_size=null \
 algorithm.kl_ctrl.kl_coef=0.001 \
 +trainer.val_before_train=False \
 trainer.default_hdfs_dir=null \
 trainer.n_gpus_per_node=1 \
 trainer.nnodes=1 \
 trainer.save_freq=10 \
 trainer.test_freq=10 \
 trainer.total_epochs=5 \
 trainer.logger=\[console\]

2025-05-14 12:14:05,782	INFO worker.py:1879 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(TaskRunner pid=155984) {'actor_rollout_ref': {'actor': {'clip_ratio': 0.2,
(TaskRunner pid=155984)                                  'entropy_coeff': 0.001,
(TaskRunner pid=155984)                                  'fsdp_config': {'fsdp_size': -1,
(TaskRunner pid=155984)                                                  'optimizer_offload': False,
(TaskRunner pid=155984)                                                  'param_offload': False,
(TaskRunner pid=155984)                                                  'wrap_policy': {'min_num_params': 0}},
(TaskRunner pid=155984)                                  'grad_clip': 1.0,
(TaskRunner pid=155984)                                  'kl_loss_coef': 0.001,
(TaskRunner pid=155984)                                  'kl_loss_type': 'low_var_kl',
(TaskRunner pid=155984)                                  'optim': {'lr': 1e-06,

In [ ]:
actor_model.save_pretrained("/home/warp/models/fine_tuned_model")


# Stop and clean up resources

In [12]:
!ray stop

Did not find any active Ray processes.
